In [1]:
%%capture output 
# captures ALL output in cell to disable tensorflow warnings

import numpy as np
from stable_baselines import PPO1

In [2]:
import sys
sys.path.insert(0,'../backend')

# load our webotsgym
import webotsgym as wg

In [3]:
class MyEval(wg.WbtReward):
    def __init__(self, env, config: wg.WbtConfig = wg.WbtConfig()):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        """calculate the reward of the current state
           Returns: (double) value of reward
        """
        pass
    
    def check_done(self):
        """check whether this episode should be ended or not
           Returns: (bool) end this episode or not
        """
        pass

In [4]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.RUN

In [5]:
# normal
env = wg.WbtGym(config=config)

Accepting on Port:  10201


In [6]:
# custom reward class
# env = wg.WbtGym(config=config,
#                     evaluate_class=MyEval)

In [7]:
%%capture output 
# captures ALL output in cell to disable tensorflow warnings

model_name = "PPO_webots_continuous"
model = PPO1("MlpPolicy", env)

In [8]:
time_steps = 1000
model.learn(total_timesteps=time_steps)
model.save("model/continuous/{}".format(model_name))